In [727]:
import psycopg2
import pandas as pd
from langchain.tools import tool
from urllib.parse import quote_plus


In [730]:
from langchain.tools import tool
@tool
def getAllItems():
    '''
    This function used to fetch all the item details. Below are the schema of the table name dim_products
    
        product_name text  - This field has the item name 
        product_id bigint  - This field has the product ID or SKU ID
        category text      - This field has the product Catagory
        "price_INR" bigint - This field has the product Price in INR
        "price_USD" double precision -- This field has the product Price in USD
    '''
    conn = createSupabaseConnection()
    query = "SELECT * FROM dim_products;"
    df = pd.read_sql(query, conn)
    conn.close()
    return df


In [731]:
@tool
def getAllOrderLines():
    '''
    This function used to fetch all the order details. Below are the schema of the table name fact_aggregate

            order_id text               - This Holds the Order number 
            customer_id bigint          - Customer ID
            order_placement_date date   - Order placed date
            on_time bigint              - If the order delivered on time 
            in_full text                - If the order shipped fully or partial of the order fulfilled 
            otif text                   - On time %
    '''
    conn = createSupabaseConnection()
    print("connection from getOrderLines --> ", conn)
    query = "SELECT * FROM fact_aggregate;"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [732]:
@tool
def getAllOrders():
    '''
    This function used to fetch all the order line details. Below are the schema of the table name fact_order_line

            order_id text               - This Holds the Order number
            order_placement_date date   - This holds the order Placement date
            customer_id bigint          - This HOlds the customer id
            product_id bigint           - This holds the Product ID or customer ID 
            order_qty bigint            - This holds the ordered Quantity of that line
            agreed_delivery_date date   - This holds the promise date
            actual_delivery_date date   - This holds the actual delivery date
            delivery_qty bigint         - This holds how many quantity has been shipped
            "In Full" text              - Is the order shipped fully or there is a cancellation (Partial/Full) 
            "On Time" text              - This holds the order fulfilled on time or not
            "On Time In Full" text      -This holds the order fulfilled on time or not, and all the quantity shipped fully or not
    '''
    conn = createSupabaseConnection()
    query = "SELECT * FROM fact_order_line;"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [733]:
import re

def extract_sql_from_text(text: str) -> str:
    """
    Extracts the first SQL code block from the LLM response.
    """
    match = re.search(r"```sql\n(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # Fallback: return everything from SELECT onward
        select_pos = text.lower().find("select")
        return text[select_pos:].strip() if select_pos != -1 else text

In [734]:
@tool
def executeQuery(query: str):
    '''
    This function used to fetch all data based on the query
    '''
    final_query = extract_sql_from_text(query)
    conn = createSupabaseConnection()
    print("Query from executeQuery -->", query)
    df = pd.read_sql(final_query, conn)
    conn.close()
    #return df
    return df.to_markdown(index=False, tablefmt="retaildata")


In [735]:
custom_prompt_str = '''
You are a Helpful assistant who can create SQL Query Keeping in mind the database is in postgresql.
You can create the SImple as well as the complex Queries as well. Below are the tables-
- dim_products
- fact_order_line
- fact_aggregate

Also Below is the table structure and there fields - 
dim_products -
        product_name text  - This field has the item name 
        product_id bigint  - This field has the product ID or SKU ID
        category text      - This field has the product Catagory
        "price_INR" bigint - This field has the product Price in INR
        "price_USD" double precision -- This field has the product Price in USD

fact_order_line -
        order_id text               - This Holds the Order number
        order_placement_date date   - This holds the order Placement date
        customer_id bigint          - This HOlds the customer id
        product_id bigint           - This holds the Product ID or customer ID 
        order_qty bigint            - This holds the ordered Quantity of that line
        agreed_delivery_date date   - This holds the promise date
        actual_delivery_date date   - This holds the actual delivery date
        delivery_qty bigint         - This holds how many quantity has been shipped
        "In Full" text              - Is the order shipped fully or there is a cancellation (Partial/Full) 
        "On Time" text              - This holds the order fulfilled on time or not
        "On Time In Full" text      -This holds the order fulfilled on time or not, and all the quantity shipped fully or not

fact_aggregate -
        order_id text               - This Holds the Order number 
        customer_id bigint          - Customer ID
        order_placement_date date   - Order placed date
        on_time bigint              - If the order delivered on time 
        in_full text                - If the order shipped fully or partial of the order fulfilled 
        otif text                   - On time %

Relation between the tables-
dim_products has an One to One relationship with fact_order_line on product_id
fact_order_line has an One to One relationship with fact_aggregate on order_id

Rules:
        - Only give the output in a SQL formet that is ready to run.
        - Do not include any other tables and columns other that that.
        - All the dates are in YYYY-MM-DD format and are in Date datatype

Input: Give me the orders placed for item 25891503
Output: SELECT 
    fol.order_id,
    fol.order_placement_date,
    fol.customer_id,
    fol.order_qty,
    fol.agreed_delivery_date,
    fol.actual_delivery_date,
    fol.delivery_qty,
    fol."In Full",
    fol."On Time",
    fol."On Time In Full"
FROM 
    fact_order_line fol
JOIN 
    dim_products dp ON fol.product_id = dp.product_id
WHERE 
    dp.product_id = 25891503;


'''

In [736]:
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"],
    template = '''
    You are a Helpful assistant who can create SQL Query Keeping in mind the database is in postgresql.
    You have access to the below tools-
    {tools}
    You can create the Simple as well as the complex Queries as well. Below are the tables-
    - dim_products
    - fact_order_line
    - fact_aggregate

    Also Below is the table structure and there fields - 
    dim_products -
            product_name text  - This field has the item name 
            product_id bigint  - This field has the product ID or SKU ID
            category text      - This field has the product Catagory
            "price_INR" bigint - This field has the product Price in INR
            "price_USD" double precision -- This field has the product Price in USD

    fact_order_line -
            order_id text               - This Holds the Order number
            order_placement_date date   - This holds the order Placement date
            customer_id bigint          - This HOlds the customer id
            product_id bigint           - This holds the Product ID or customer ID 
            order_qty bigint            - This holds the ordered Quantity of that line
            agreed_delivery_date date   - This holds the promise date
            actual_delivery_date date   - This holds the actual delivery date
            delivery_qty bigint         - This holds how many quantity has been shipped
            "In Full" text              - Is the order shipped fully or there is a cancellation (Partial/Full) 
            "On Time" text              - This holds the order fulfilled on time or not
            "On Time In Full" text      -This holds the order fulfilled on time or not, and all the quantity shipped fully or not

    fact_aggregate -
            order_id text               - This Holds the Order number 
            customer_id bigint          - Customer ID
            order_placement_date date   - Order placed date
            on_time bigint              - If the order delivered on time 
            in_full text                - If the order shipped fully or partial of the order fulfilled 
            otif text                   - On time %

    Relation between the tables-
    dim_products has an One to One relationship with fact_order_line on product_id
    fact_order_line has an One to One relationship with fact_aggregate on order_id

    Rules:
            - Only give the output in a SQL formet that is ready to run.
            - Do not include any other tables and columns other that that.
            - All the dates are in YYYY-MM-DD format and are in Date datatype
            - Use tools when needed (from: {tool_names}).

    Input: Give me the orders placed for item 25891503
    Output: SELECT 
        fol.order_id,
        fol.order_placement_date,
        fol.customer_id,
        fol.order_qty,
        fol.agreed_delivery_date,
        fol.actual_delivery_date,
        fol.delivery_qty,
        fol."In Full",
        fol."On Time",
        fol."On Time In Full"
    FROM 
        fact_order_line fol
    JOIN 
        dim_products dp ON fol.product_id = dp.product_id
    WHERE 
        dp.product_id = 25891503;

    Question: {input}
    
    {agent_scratchpad}
    ''')

In [737]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        ("system",custom_prompt_str),
        ("user","{input}" )
    ]
)
print(prompt.input_variables)
print(prompt.output_parser)

['input']
None


In [738]:
tools = [executeQuery]

In [740]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [741]:
response = prompt | llm
output=response.invoke({"input":"Show me KPI's of Orders on 17th Jan 2025"})
print(output.content)
print("from extract_sql_from_text \n",extract_sql_from_text(output.content))
query_output = extract_sql_from_text(output.content)

SELECT 
    fo.order_id,
    fo.customer_id,
    fo.order_placement_date,
    fa.on_time,
    fa.in_full,
    fa.otif
FROM 
    fact_aggregate fa
JOIN 
    fact_order_line fo ON fa.order_id = fo.order_id
WHERE 
    fo.order_placement_date = '2025-01-17';
from extract_sql_from_text 
 SELECT 
    fo.order_id,
    fo.customer_id,
    fo.order_placement_date,
    fa.on_time,
    fa.in_full,
    fa.otif
FROM 
    fact_aggregate fa
JOIN 
    fact_order_line fo ON fa.order_id = fo.order_id
WHERE 
    fo.order_placement_date = '2025-01-17';


In [742]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

prompt_hub = hub.pull("hwchase17/react")



In [743]:

agent = create_react_agent(
    llm=llm,
    tools = tools,
    prompt= prompt_hub
)

In [686]:
agent_executor = AgentExecutor(
    agent = agent,
    tools= tools,
    verbose=True
)

In [ ]:
print(query_output)
response = agent_executor.invoke({"input":"Show me KPI's of Orders on 17th Jan 2025", "query":output.content})
print(agent_executor.input_keys)
print(response["output"])

SELECT 
    fa.order_id,
    fa.customer_id,
    fa.on_time,
    fa.in_full,
    fa.otif
FROM 
    fact_aggregate fa
WHERE 
    fa.order_placement_date = '2025-01-17';


> Entering new AgentExecutor chain...
The question is asking for specific data from the `fact_aggregate` table. The query is specifically designed to fetch details about orders from a particular date, namely '2025-01-17'. To provide an answer, I need to execute the query against the database to retrieve the required data.

Action: executeQuery
Action Input: 
```sql
SELECT 
    fa.order_id,
    fa.customer_id,
    fa.on_time,
    fa.in_full,
    fa.otif
FROM 
    fact_aggregate fa
WHERE 
    fa.order_placement_date = '2025-01-17';
```conn --> <connection object at 0x113fa1e70; dsn: 'user=postgres.fxpuwjgwfhkapgjinyss password=xxx dbname=postgres host=aws-0-us-east-2.pooler.supabase.com port=5432 sslmode=require', closed: 0>
Query from executeQuery --> ```sql
SELECT 
    fa.order_id,
    fa.customer_id,
    fa.on_time,
 

/var/folders/7_/vzxv0s1j43n3s8qd35q3b6_m0000gn/T/ipykernel_6909/3163670763.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(final_query, conn)


Given the data retrieved from the `fact_aggregate` table for orders placed on '2025-01-17', I can now provide the details requested. The query successfully fetched information such as `order_id`, `customer_id`, `on_time`, `in_full`, and `otif` for orders placed on the specified date.

Final Answer: The query successfully retrieved data from the `fact_aggregate` table. The data includes various orders along with their `order_id`, `customer_id`, and performance metrics (`on_time`, `in_full`, `otif`). If you need further analysis or a specific summarization, please let me know!

> Finished chain.
[]
The query successfully retrieved data from the `fact_aggregate` table. The data includes various orders along with their `order_id`, `customer_id`, and performance metrics (`on_time`, `in_full`, `otif`). If you need further analysis or a specific summarization, please let me know!
